<font size = "6"> Fordham Sports Analytics Society Big Data Bowl 2023 - Exploratory Analysis </font>

<font size = "4"> Run through ideas for eventual model, create new model features, and understand provided materials. </font>

- Authors:  Peter Majors, Chris Orlando, and Jack Townsend
- Kaggle:  https://www.kaggle.com/competitions/nfl-big-data-bowl-2023/overview (Resources)
- Our Github:  https://github.com/peterlmajors/FSAS_BigDataBowl_2023 (Up-To-Date Code)

In [1]:
#Import Required Packages

#Data Manipulation
import pandas as pd
import numpy as np
import math

#Data Vizualization
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from haversine import haversine

#Notebook Settings
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
#Importing Kaggle Data (Needed For Animated Plays Function (Takes Original Data))

# #Games - Basic Information On All Games
games = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/games.csv")

# #pffScout - PFF Judgements For Each Player On Each Play
pffScout = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/pffScoutingData.csv")

# #Players - Basics On Players
players = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/players.csv")

# #Plays - Everthing About Specific Plays
plays = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/plays.csv")

# #Week - Frame-By-Frame Player Tracking
week1 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week1.csv")
week2 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week2.csv")
week3 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week3.csv")
week4 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week4.csv")
week5 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week5.csv")
week6 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week6.csv")
week7 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week7.csv")
week8 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week8.csv")

In [3]:
#Define Function To Animate Plays (Needed)
colors = {'ARI':"#97233F", 'ATL':"#A71930", 'BAL':'#241773', 'BUF':"#00338D", 'CAR':"#0085CA", 'CHI':"#C83803", 'CIN':"#FB4F14", 'CLE':"#311D00", 
          'DAL':'#003594', 'DEN':"#FB4F14", 'DET':"#0076B6", 'GB':"#203731", 'HOU':"#03202F", 'IND':"#002C5F", 'JAX':"#9F792C", 'KC':"#E31837", 
          'LA':"#003594", 'LAC':"#0080C6", 'LV':"#000000", 'MIA':"#008E97", 'MIN':"#4F2683", 'NE':"#002244", 'NO':"#D3BC8D", 'NYG':"#0B2265", 
          'NYJ':"#125740", 'PHI':"#004C54", 'PIT':"#FFB612", 'SEA':"#69BE28", 'SF':"#AA0000", 'TB':'#D50A0A', 'TEN':"#4B92DB", 'WAS':"#5A1414", 
          'football':'#CBB67C'}

def animate_play(tracking_df, play_df,players,pffScoutingData, gameId,playId):
    selected_play_df = play_df[(play_df.playId==playId)&(play_df.gameId==gameId)].copy()
    
    tracking_players_df = pd.merge(tracking_df,players,how="left",on = "nflId")
    tracking_players_df = pd.merge(tracking_players_df,pffScoutingData,how="left",on = ["nflId","playId","gameId"])
    selected_tracking_df = tracking_players_df[(tracking_players_df.playId==playId)&(tracking_players_df.gameId==gameId)].copy()

    sorted_frame_list = selected_tracking_df.frameId.unique()
    sorted_frame_list.sort()

    # get play General information 
    line_of_scrimmage = selected_play_df.absoluteYardlineNumber.values[0]
    first_down_marker = line_of_scrimmage + selected_play_df.yardsToGo.values[0]
    down = selected_play_df.down.values[0]
    quarter = selected_play_df.quarter.values[0]
    gameClock = selected_play_df.gameClock.values[0]
    playDescription = selected_play_df.playDescription.values[0]
    # Handle case where we have a really long Play Description and want to split it into two lines
    if len(playDescription.split(" "))>15 and len(playDescription)>115:
        playDescription = " ".join(playDescription.split(" ")[0:16]) + "<br>" + " ".join(playDescription.split(" ")[16:])

    # initialize plotly start and stop buttons for animation
    updatemenus_dict = [
        {
             "buttons": [
                {
                    "args": [None, {"frame": {"duration": 100, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 0}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]
    # initialize plotly slider to show frame position in animation
    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }


    frames = []
    for frameId in sorted_frame_list:
        data = []
        # Add Numbers to Field 
        data.append(
            go.Scatter(
                x=np.arange(20,110,10), 
                y=[5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        data.append(
            go.Scatter(
                x=np.arange(20,110,10), 
                y=[53.5-5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add line of scrimage 
        data.append(
            go.Scatter(
                x=[line_of_scrimmage,line_of_scrimmage], 
                y=[0,53.5],
                line_dash='dash',
                line_color='blue',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add First down line 
        data.append(
            go.Scatter(
                x=[first_down_marker,first_down_marker], 
                y=[0,53.5],
                line_dash='dash',
                line_color='yellow',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Plot Players
        for team in selected_tracking_df.team.unique():
            plot_df = selected_tracking_df[(selected_tracking_df.team==team)&(selected_tracking_df.frameId==frameId)].copy()
            if team != "football":
                hover_text_array=[]
                for nflId in plot_df.nflId:
                    selected_player_df = plot_df[plot_df.nflId==nflId]
                    hover_text_array.append("nflId:{}<br>displayName:{}<br>Position:{}<br>Role:{}".format(selected_player_df["nflId"].values[0],
                                                                                      selected_player_df["displayName"].values[0],
                                                                                      selected_player_df["pff_positionLinedUp"].values[0],
                                                                                      selected_player_df["pff_role"].values[0]))
                data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color=colors[team],name=team,hovertext=hover_text_array,hoverinfo="text"))
            else:
                data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color=colors[team],name=team,hoverinfo='none'))
        # add frame to slider
        slider_step = {"args": [
            [frameId],
            {"frame": {"duration": 100, "redraw": False},
             "mode": "immediate",
             "transition": {"duration": 0}}
        ],
            "label": str(frameId),
            "method": "animate"}
        sliders_dict["steps"].append(slider_step)
        frames.append(go.Frame(data=data, name=str(frameId)))

    scale=10
    layout = go.Layout(
        autosize=False,
        width=120*scale,
        height=60*scale,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        # Create title and add play description at the bottom of the chart for better visual appeal
        title=f"GameId: {gameId}, PlayId: {playId}<br>{gameClock} {quarter}Q"+"<br>"*19+f"{playDescription}",
        updatemenus=updatemenus_dict,
        sliders = [sliders_dict]
    )

    fig = go.Figure(
        data=frames[0]["data"],
        layout= layout,
        frames=frames[1:]
    )
    # Create First Down Markers 
    for y_val in [0,53]:
        fig.add_annotation(
                x=first_down_marker,
                y=y_val,
                text=str(down),
                showarrow=False,
                font=dict(
                    family="Courier New, monospace",
                    size=16,
                    color="black"
                    ),
                align="center",
                bordercolor="black",
                borderwidth=2,
                borderpad=4,
                bgcolor="#ff7f0e",
                opacity=1
                )

    return fig

In [4]:
#Import Our Merged DataFrames (Needed)
ptrack = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/ptrack.csv")
pbp = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/pbp.csv")

<font size="5">Check Out Individual Plays</font>

In [13]:
#Choose A Game and Play
gameId = 2021090900
playId = 456

#Run Animate Plays Function
animate_play(week1, plays, players, pffScout, gameId, playId)

In [14]:
#Print Out All Frames From A Player and Play
ptrack.loc[(ptrack['gameId'] == gameId) & (ptrack['nflId'] == 42377) & (ptrack['playId'] == playId)].sort_values(by = 'frameId', ascending = True).head(100)

#Print Out All Frames From Selected Game
#ptrack_sample = ptrack.loc[(ptrack['gameId'] == gameId)]

#Print Out All Plays From Selected Game
#ptrack.loc[(ptrack.gameId == gameId) & (ptrack.playDirection == 'right')]['playId'].sort_values().unique()

#Print Out All pbp Data From Selected Play
#pbp[(pbp['gameId'] == gameId) & (pbp['playId'] == playId)]

#Print Out All pffScout Data From Selected Play
#pffScout.loc[(pffScout['gameId'] == gameId) & (pffScout['playId'] == playId)]

,Unnamed: 0,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,height,weight,birthDate,collegeName,officialPosition,displayName,x_qb,y_qb,s_qb,a_qb,dis_qb,o_qb,dir_qb,dist_from_qb,angle_to_qb,angle_to_qb_diff_o,nflId_blocker,rusher_in_imm_box
2162,2162,2021090900,456,42377,Pass Block,LT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42403.0,PP,0.0,1,2021-09-10T00:39:33.700,76.0,TB,right,29.25,30.28,0.00,0.00,0.00,95.67,287.72,None,6-6,338,1993-06-23,Penn State,T,Donovan Smith,25.92,27.34,0.0,0.0,0.0,120.91,73.87,4.442128,52.064220,NaN,42377.0,False
2184,2184,2021090900,456,42377,Pass Block,LT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42403.0,PP,0.0,2,2021-09-10T00:39:33.800,76.0,TB,right,29.25,30.27,0.00,0.00,0.00,95.67,271.26,None,6-6,338,1993-06-23,Penn State,T,Donovan Smith,25.93,27.34,0.0,0.0,0.0,123.74,76.76,4.428013,52.086423,NaN,42377.0,False
2206,2206,2021090900,456,42377,Pass Block,LT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42403.0,PP,0.0,3,2021-09-10T00:39:33.900,76.0,TB,right,29.24,30.27,0.03,0.42,0.00,94.94,233.38,None,6-6,338,1993-06-23,Penn State,T,Donovan Smith,25.93,27.34,0.0,0.0,0.0,125.16,78.43,4.420520,51.918723,NaN,42377.0,False
2228,2228,2021090900,456,42377,Pass Block,LT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42403.0,PP,0.0,4,2021-09-10T00:39:34.000,76.0,TB,right,29.24,30.27,0.11,0.81,0.01,94.94,256.82,None,6-6,338,1993-06-23,Penn State,T,Donovan Smith,25.93,27.34,0.0,0.0,0.0,125.16,75.17,4.420520,51.918723,NaN,42377.0,False
2250,2250,2021090900,456,42377,Pass Block,LT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42403.0,PP,0.0,5,2021-09-10T00:39:34.100,76.0,TB,right,29.21,30.28,0.31,1.29,0.03,92.67,274.83,None,6-6,338,1993-06-23,Penn State,T,Donovan Smith,25.92,27.34,0.0,0.0,0.0,125.16,70.88,4.412222,51.390837,NaN,42377.0,False


<font size="5">Explore Effect Of Block Types On Hurry Rate</font>

In [ ]:
#For Each Block Type, Find The Success Rate of A Defesnvie Player Creating A Pressure

# #How We Found Positions
# #pffScout.pff_positionLinedUp.sort_values().unique()

# #Create A New Data Frame Only With 9 D-Linemen Positons
# ptrackl = ptrack.loc[(ptrack['pff_positionLinedUp'] == "LOLB") | (ptrack['pff_positionLinedUp'] == "ROLB") | (ptrack['pff_positionLinedUp'] == "LEO") |
#     (ptrack['pff_positionLinedUp'] == "REO") | (ptrack['pff_positionLinedUp'] == "DLT") | (ptrack['pff_positionLinedUp'] == "DRT") |
#     (ptrack['pff_positionLinedUp'] == "NLT") | (ptrack['pff_positionLinedUp'] == "NRT") | (ptrack['pff_positionLinedUp'] == "NT")]

# #Must First Change The pff_blockType in ptrack To A String (Data Did Not Group By Properly When An Int)
# ptrackl.pff_blockType = ptrack.pff_blockType.astype("str")

# #For Each Block Type, Find The Percentage of Time A Hury Occurs
# ptrackl.groupby('pff_blockType')['pff_hurry'].count() #Ideally Would Be .average() Instead

# #Frequency of Different Type of Blocks Performed (EDA)
# #ptrack.pff_blockType.value_counts()

# #Issue Is Only The Players Issuing The Block (Offensive Players) Are Assigned A Block Type
# #However, The Defensive Players They Blocked Are Noted In The pff_nflIdBlockedPlayer Column In ptrack

In [ ]:
# #Create A Data Frame Containing Only Plays Where A Pass Block Occured (One Row Per Block)
# ptrack_blocks = ptrack.loc[ptrack['pff_blockType'] != '0'][['gameId', 'playId', 'nflId', 'pff_nflIdBlockedPlayer', 'pff_blockType', 'pff_hurryAllowed']]

# #Reduce To Each Play (One Row Per Interaction Between Blocked Player And Blocker) (First Interation of A Defensive Player Being Blocked In Play)
# ptrack_blocks = ptrack_blocks.drop_duplicates() 

# #Sampler of The ptrack_blocks Data Frame
# ptrack_blocks.sort_values('gameId', ascending = False).head(5)

# #Show How Often Each Block Type Resulted In A Hurry
# ptrack_block_hurry = pd.DataFrame(ptrack_blocks.groupby('pff_blockType')['pff_hurryAllowed'].mean().round(4).sort_values(ascending = False))
# ptrack_block_hurry

In [ ]:
# #Can Each Offensive Player Only Get One Block Per Play (I'd Assume Not, But Lets Check) (Yes They Do!)
# ptrack_blocks_per_play = pd.DataFrame(ptrack_blocks.groupby(['gameId', 'playId', 'nflId'])['pff_blockType'].count())
# ptrack_blocks_per_play = ptrack_blocks_per_play.reset_index()

# ptrack_blocks_per_play[ptrack_blocks_per_play['pff_blockType'] != 1].head() #No Rows Appear
# #Yes! Each Offensive Player Only Gets One Block Per Play!

<font size="5"> Explore  Distance From Quarterback At Each Second </font>

Each D-Line Position and Each Second Post Snap

In [ ]:
#Create Role-Specific Data Frames
ptrack_prush = ptrack[ptrack.pff_role == "Pass Rush"]
ptrack_pblock = ptrack[ptrack.pff_role == "Pass Block"]

#Descriptive Stats On Frames In Plays
pd.DataFrame(ptrack.groupby(['gameId', 'playId'])['frameId'].max()).mean()

#Available Roles On The Field
ptrack.pff_role.value_counts()

#Top 8 Most Popular Pass Rush Positions
ptrack_prush.pff_positionLinedUp.value_counts().reset_index().head(8)

In [ ]:
#Display Scatter Plot
ptrack_prush_drt_dist_from_qb = ptrack_prush[ptrack_prush['pff_positionLinedUp'] == 'RE'].groupby('frameId')['dist_from_qb'].mean().reset_index()
plt.scatter(x = ptrack_prush_drt_dist_from_qb.frameId, y = ptrack_prush_drt_dist_from_qb.dist_from_qb, marker = "8")
plt.title("Average Distance From The Quarterback As Time Progresses (RE)")
plt.xlabel("Frames (10 Frames = 1 Second Post-Snap)")
plt.ylabel("Euclidead Distance From Quarterback")

<font size="5"> Explore Angle of Offensive Linemen Relative To Direction of Pass Rusher During Contact </font>

In [ ]:
#While Engaged In The Immediate Zone, How Do Shoulder Angles Change?

<font size="5"> Define The "Immediate Zone" Depth For Pass Blockers </font>

In [ ]:
#Gather Data And Fields Relevant To Answering The Question

#Crate Data Frame With Only Pass Blocking Plays
ptrack_block = ptrack.loc[ptrack['pff_blockType'] != '0']

#Find Rows With Players Who Were Blocked Against (And Who Have The Role of Pass Rusher)
ptrack_block_rushers = ptrack.loc[(ptrack.nflId.isin(ptrack_block.pff_nflIdBlockedPlayer)) & (ptrack.pff_role == "Pass Rush")]
ptrack_block_rushers = ptrack_block_rushers[['gameId', 'playId', 'nflId', 'frameId', 'pff_role', 'pff_positionLinedUp', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'displayName']]

#Merge Pass Blocking Plays Data From O-Linemen Perspecitve With Pass Rusher Tracking Data
ptrack_imm_box = ptrack_block.merge(ptrack_block_rushers, left_on = ['gameId', 'playId', 'pff_nflIdBlockedPlayer', 'frameId'], 
                                                        right_on = ['gameId', 'playId', 'nflId', 'frameId'], how = 'inner')

#Reduce To Columns Of Interest
ptrack_imm_box = ptrack_imm_box[['gameId', 'playId', 'nflId_x', 'frameId', 'pff_role_x', 'pff_positionLinedUp_x', 'pff_blockType','x_x', 'y_x', 's_x', 'a_x', 'dis_x', 'o_x', 'dir_x', 
            'displayName_x','nflId_y', 'pff_role_y', 'pff_positionLinedUp_y', 'x_y', 'y_y', 's_y', 'a_y', 'dis_y', 'o_y', 'dir_y', 'displayName_y']]

#Rename Columns
ptrack_imm_box = ptrack_imm_box.rename(columns = {"nflId_x":"nflId_blocker", "displayName_x": "displayName_blocker", "pff_role_x": "pff_role_blocker", 
            "pff_positionLinedUp_x": "pff_positionLinedUp_blocker", "x_x": "x_blocker", "y_x": "y_blocker", "s_x": "s_blocker", "a_x": "a_blocker", "dis_x": "dis_blocker", 
            "o_x": "o_blocker", "dir_x": "dir_blocker", "nflId_y":"nflId_rusher", "displayName_y": "displayName_rusher", "pff_role_y": "pff_role_rusher", 
            "pff_positionLinedUp_y": "pff_positionLinedUp_rusher", "x_y": "x_rusher", "y_y": "y_rusher", "s_y": "s_rusher", "a_y": "a_rusher", "dis_y": "dis_rusher", 
            "o_y": "o_rusher", "dir_y": "dir_rusher"})

#Calculate Distance Between Pass Blocker And Pass Rusher at Each Frame
ptrack_imm_box['blocker_rusher_distance'] = np.hypot((ptrack_imm_box.y_blocker - ptrack_imm_box.y_rusher), (ptrack_imm_box.x_blocker - ptrack_imm_box.x_rusher))

#Calculate Difference Between Rusher Direction and Blocker Orientation
ptrack_imm_box['diff_btw_rusher_dir_blocker_o'] = abs((ptrack_imm_box.dir_rusher - 180) - ptrack_imm_box.o_blocker)

In [ ]:
#How Far Away From A Blocker Does A Pass Rusher Slow Down or Deccelerate? (1 Yard In Front and 1.5 Yards Across (.75 Either Side))
#Filter Down To Where Rusher Direction and Blocker Orientation Are Generally Matched (25 Degrees In Either Direction)
ptrack_imm_box_10orless = ptrack_imm_box[ptrack_imm_box.diff_btw_rusher_dir_blocker_o < 10]

#Speed
#distance_and_speed = pd.DataFrame(ptrack_imm_box_10orless.groupby(round(ptrack_imm_box_10orless.blocker_rusher_distance,0))['s_rusher'].mean()).reset_index()
#sns.scatterplot(distance_and_speed.blocker_rusher_distance, distance_and_speed.s_rusher)

#Acceleration
distance_and_acceleration = pd.DataFrame(ptrack_imm_box_10orless.groupby(round(ptrack_imm_box_10orless.blocker_rusher_distance,0))['a_rusher'].mean()).reset_index()
sns.scatterplot(distance_and_acceleration.blocker_rusher_distance, distance_and_acceleration.a_rusher)

#Acceleration Hit Their Lowest Rates Within 1-2 Yards of The Pass Blocker!!

<font size="5"> Explore The Immediate Zone At Each Frame For Pass Blockers </font>

In [ ]:
#Create The Immediate Zone In The Main ptrack Data Frame 

#Define The Width And Depth Of This Zone
imm_box_width = .75
imm_box_depth = 1

#bl = Bottom Left
ptrack['bl_rz_x'] = ptrack.x - (imm_box_width * np.cos(np.radians(ptrack.o)))
ptrack['bl_rz_y'] = ptrack.y + (imm_box_width * np.sin(np.radians(ptrack.o)))

#br = Bottom Right
ptrack['br_rz_x'] = ptrack.x + (imm_box_width * np.cos(np.radians(ptrack.o)))
ptrack['br_rz_y'] = ptrack.y - (imm_box_width * np.sin(np.radians(ptrack.o)))

#fl = Front Left
ptrack['fl_rz_x'] = ptrack.bl_rz_x + (imm_box_depth * np.cos(np.radians(ptrack.o - 90)))
ptrack['fl_rz_y'] = ptrack.bl_rz_y - (imm_box_depth * np.sin(np.radians(ptrack.o - 90)))

#fr = Front Right
ptrack['fr_rz_x'] = ptrack.br_rz_x + (imm_box_depth * np.cos(np.radians(ptrack.o - 90)))
ptrack['fr_rz_y'] = ptrack.br_rz_y - (imm_box_depth * np.sin(np.radians(ptrack.o - 90)))


In [ ]:
#Plot Instances of Rushers In A Blocker's Immediate Zone

#Provide Orientation Angle (Whole Number)
o_angle = 50

#In Order To Test If The Above Calculations Are Correct, First Find Plays With A Known Orientation Angle (Choose One)
ptrack_o_filter = ptrack.loc[(ptrack.gameId == 2021090900) & (round(ptrack.o,0) == o_angle)].head(1).reset_index()

#Use The Player, Play, and Frame To Create A Scatter Plot With The Immediate Zone
ptrack_filter = ptrack.loc[(ptrack.gameId == ptrack_o_filter.at[0, 'gameId']) & (ptrack.playId == ptrack_o_filter.at[0, 'playId']) & (ptrack.nflId == ptrack_o_filter.at[0, 'nflId']) & (ptrack.frameId == ptrack_o_filter.at[0, 'frameId'])].reset_index()

#Plot The Immediate Zone On A Play
sns.scatterplot(data = ptrack_filter, x = 'bl_rz_x', y = 'bl_rz_y', marker = '^', s = 200)
sns.scatterplot(data = ptrack_filter, x = 'br_rz_x', y = 'br_rz_y', marker = '^', s = 200)
sns.scatterplot(data = ptrack_filter, x = 'fl_rz_x', y = 'fl_rz_y', marker = 'v', s = 200)
sns.scatterplot(data = ptrack_filter, x = 'fr_rz_x', y = 'fr_rz_y', marker = 'v', s = 200)
sns.scatterplot(x = ptrack_filter.x, y = ptrack_filter.y, marker = 'o', s = 100)
plt.scatter(x = 41.5, y = 21.75, s = 100, marker = 'X')
print("Provided Player Orientation In Degrees (Second Number): ", ptrack_filter['o'])

<font size="5"> Understand The Immediate Zone On Plays Where Blocks Are Known To Occur </font>

In [ ]:
ptrack.loc[(ptrack.pff_blockType != 0) & (ptrack.pff_role == "Pass Rush")].rusher_in_imm_box.value_count()

<font size="5"> Explore Events Classifications (Understand Beginning and End Of QB Possession) </font>

Starting at: ball_snap

Ending At: qb_sack, pass_forward, run, 

Bookmarked: first_contact, 

In [ ]:
#Determine Which Events Are The End Of QB Possession
ptrack.event.value_counts()

In [ ]:
#Use Event Classification To Find Play Information In pbp Data
ptrack_fc = ptrack[ptrack.event == 'first_contact']
pbp_fc = pbp.merge(ptrack_fc, how = 'inner', on = ['gameId', 'playId'])
pbp_fc.drop_duplicates(['gameId', 'playId'])

#Additional Searches
# ptrack.loc[(ptrack.gameId == 2021091204) & (ptrack.playId == 1979) & (ptrack.pff_positionLinedUp == 'QB')] 

<font size="5"> Explore Drop Back Types </font>

In [ ]:
#Frequency of Drop Backs Types
pbp.dropBackType.value_counts()

In [ ]:
#How Is Drop Back Type Influenced By Other Factors
pbp.groupby('dropBackType')['defendersInBox'].mean().sort_values()